<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/PyTorchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This* NoteBook illustrates the examples given in chapter 1 of Inside Deep Learning Book.

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


Any value of n for an n-dimensional array is still a tensor. The word tensor is simply referring to the overall concept of an n-dimenional array.
We care about the shape of the tensor as we use different shapes to represent different types of data.

In [3]:
import torch

torch_scalar = torch.tensor(3.14)
torch_vector = torch.tensor([1, 2, 3, 4])
torch_matrix = torch.tensor([[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12] 
                            ])
torch_tensor3d = torch.tensor([ 
                               [[1, 2, 3, 4],
                                [5, 6, 7, 8],
                                [9, 10, 11, 12] 
                               ],
                               [[13, 14, 15, 16],
                                [17, 18, 19, 20],
                                [21, 22, 23, 24] 
                               ]
                              ])

print(torch_scalar.shape)
print(torch_vector.shape)
print(torch_matrix.shape)
print(torch_tensor3d.shape)

torch.Size([])
torch.Size([4])
torch.Size([3, 4])
torch.Size([2, 3, 4])


Converion of data from numpy to Tensor, Viceversa 
Numpy defaults to 64 bit. Pytorch defaults to 32-bit floats.
But when we create 'PyTorch Tensor' from 'Numpy Array' - Tensor will use same data type as Numpy Array 



In [4]:
x_np = np.random.random((4, 4))
print(x_np)

[[0.59372321 0.6734427  0.42833226 0.72437014]
 [0.67264186 0.81100443 0.30427199 0.07268613]
 [0.64239151 0.42747158 0.32264433 0.93385973]
 [0.25830812 0.83486154 0.79678985 0.19553079]]


In [5]:
x_pt = torch.tensor(x_np)
print(x_pt)

tensor([[0.5937, 0.6734, 0.4283, 0.7244],
        [0.6726, 0.8110, 0.3043, 0.0727],
        [0.6424, 0.4275, 0.3226, 0.9339],
        [0.2583, 0.8349, 0.7968, 0.1955]], dtype=torch.float64)


To avoid issues with the types, you can always specify the data type to be used 'dtype' with the APIS.

In [6]:
print(x_np.dtype, x_pt.dtype)
x_np = np.asarray(x_pt, dtype=np.float32)
x_pt = torch.tensor(x_np, dtype=torch.float32)
print(x_np.dtype, x_pt.dtype)

float64 torch.float64
float32 torch.float32


Logical Operations & Similarity of the APIs between Numpy and Tensor.

In [7]:
b_np = (x_np > 0.5)
b_pt = (x_pt > 0.5)
print(b_np)
print(b_np.dtype)
print(b_pt)
print(b_pt.dtype)

[[ True  True False  True]
 [ True  True False False]
 [ True False False  True]
 [False  True  True False]]
bool
tensor([[ True,  True, False,  True],
        [ True,  True, False, False],
        [ True, False, False,  True],
        [False,  True,  True, False]])
torch.bool


Not always API are similar, one such example is transose which is explained as below. Please check always documention about API's functionality.

In [8]:
np.transpose(x_np)

array([[0.59372324, 0.6726419 , 0.6423915 , 0.2583081 ],
       [0.6734427 , 0.81100446, 0.42747158, 0.8348615 ],
       [0.42833227, 0.304272  , 0.32264432, 0.7967898 ],
       [0.7243701 , 0.07268614, 0.9338597 , 0.19553079]], dtype=float32)

In [9]:
torch.transpose(x_pt, 0, 1)

tensor([[0.5937, 0.6726, 0.6424, 0.2583],
        [0.6734, 0.8110, 0.4275, 0.8349],
        [0.4283, 0.3043, 0.3226, 0.7968],
        [0.7244, 0.0727, 0.9339, 0.1955]])

First important functionality of PyTorch(Tensor) beyond what numpy can provide is to use GPU to accelarate mathematical caluclations. 
Below shows some timings related to CPU and Calculations.

In [10]:
import timeit
x = torch.rand(2**11, 2**11)
time_cpu  = timeit.timeit("x@x", globals=globals(), number=100)

In [11]:
print("is CUDA available? ",torch.cuda.is_available())
device = torch.device('cuda')

x = x.to(device)
time_gpu = timeit.timeit("x@x", globals=globals(), number=100)

print(time_cpu)
print(time_gpu)

is CUDA available?  True
24.569827968000027
0.05907187500000077
